In [1]:
import os
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential
from scipy.io import wavfile as wav
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import struct
import matplotlib.pyplot as plt
import IPython.display as ipd
import progressbar
import time

In [2]:
data = pd.read_csv('metadata/Screamsound.csv')
valid_data = data[['slice_file_name', 'fold' ,'classID', 'class']][ data['end']-data['start'] > 0 ]
valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + '/' + valid_data['slice_file_name'].astype('str')


In [3]:
from tqdm import tnrange, tqdm_notebook

D=[]
        
for row in tqdm_notebook(valid_data.itertuples()): 
    print(row.path)
    print(row.classID)
    y1, sr1 = librosa.load("audio/" + row.path, duration=2.97)  
    ps = librosa.feature.melspectrogram(y=y1, sr=sr1)
    #print(ps)
    #print(ps.shape)
    if ps.shape != (128, 128): 
            continue
    D.append( (ps, row.classID) )

C:\Users\USER\AppData\Local\Temp\ipykernel_27516\2032816338.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(valid_data.itertuples()):


0it [00:00, ?it/s]

fold2/100652-3-0-0.wav
1
fold2/100652-3-0-1.wav
1
fold2/100652-3-0-2.wav
1
fold2/100652-3-0-3.wav
1
fold2/101415-3-0-2.wav
1
fold2/101415-3-0-3.wav
1
fold2/101415-3-0-8.wav
1
fold1/yisell_sound_2007_11_9_1_31_156634.wav
0
fold1/yisell_sound_2007_12_18_15_25_315033.wav
0
fold1/yisell_sound_2007_12_18_15_25_593398.wav
0
fold1/yisell_sound_2007_12_18_15_26_248752.wav
0
fold1/yisell_sound_2007_12_18_15_26_444257.wav
0
fold1/yisell_sound_2007_12_18_15_27_063699.wav
0
fold1/yisell_sound_2007_12_18_15_27_324809.wav
0
fold1/yisell_sound_2007_12_18_15_27_543156.wav
0
fold1/yisell_sound_2007_12_18_15_29_067921.wav
0
fold1/yisell_sound_2007_12_18_15_29_33722.wav
0
fold1/yisell_sound_2007_12_18_15_30_231364.wav
0
fold1/yisell_sound_2014080511023849341_66366.wav
0
fold1/yisell_sound_2014080512323863957_66366.wav
0
fold1/yisell_sound_2014081313495332740_66366.wav
0
fold1/yisell_sound_2014081314482020626_66366.wav
0
fold3/pk1e5-yogtt.wav
0
fold3/pk1e5-yogtt.wav
0
fold3/pk1e5-yogtt.wav
0
fold3/pk1e5-y

In [4]:
#import hickle as hkl 
#data = D

## Dump data to file
#hkl.dump(data, 'new_data_file.hkl')

In [5]:
## Load data from file
#D = hkl.load('new_data_file.hkl')
#print(type(D))

In [6]:
#from keras.utils import np_utils
#dataset = D
#random.shuffle(dataset)

#train = dataset[:15]
#test = dataset[15:]
#print('dataset:', len(dataset))
#print('train:', len(train))
#print('test:', len(test))

In [7]:
from keras.utils import np_utils
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split([i[0] for i in D],
                                                   [i[1] for i in D],
                                                   test_size=0.2,
                                                   random_state=0)

#X_train, y_train = zip(*train)
#X_test, y_test = zip(*test)

X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

y_train = np.array(keras.utils.np_utils.to_categorical(y_train, 10))
y_test = np.array(keras.utils.np_utils.to_categorical(y_test, 10))

In [8]:
model = Sequential()
input_shape=(128, 128, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(10))
model.add(Activation('softmax'))

In [9]:
model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=['accuracy'])

model.fit(
    x=X_train, 
    y=y_train,
    epochs=12,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
    x=X_test,
    y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('success')

Epoch 1/12
1/1 [==============================] - 1s 742ms/step - loss: 14.3897 - accuracy: 0.0667 - val_loss: 5.0499 - val_accuracy: 0.2500
Epoch 2/12
1/1 [==============================] - 0s 152ms/step - loss: 6.6223 - accuracy: 0.2667 - val_loss: 3.2034 - val_accuracy: 0.5000
Epoch 3/12
1/1 [==============================] - 0s 151ms/step - loss: 11.1051 - accuracy: 0.6000 - val_loss: 5.4924 - val_accuracy: 0.5000
Epoch 4/12
1/1 [==============================] - 0s 149ms/step - loss: 4.3994 - accuracy: 0.7333 - val_loss: 7.6812 - val_accuracy: 0.5000
Epoch 5/12
1/1 [==============================] - 0s 138ms/step - loss: 2.6805 - accuracy: 0.6667 - val_loss: 7.4876 - val_accuracy: 0.5000
Epoch 6/12
1/1 [==============================] - 0s 154ms/step - loss: 2.0390 - accuracy: 0.8667 - val_loss: 5.2998 - val_accuracy: 0.5000
Epoch 7/12
1/1 [==============================] - 0s 137ms/step - loss: 4.3816 - accuracy: 0.8000 - val_loss: 2.4467 - val_accuracy: 0.7500
Epoch 8/12
1/1 [==

In [10]:
predict = model.predict(X_test)
print(predict)

[[4.1684297e-01 5.8315694e-01 4.3327883e-19 8.2464660e-12 7.0840261e-18
  3.0325622e-18 1.5971871e-20 8.7746996e-20 1.8439356e-19 1.7176005e-22]
 [2.5022116e-01 7.4977881e-01 2.0317324e-12 6.0097785e-09 3.6744528e-13
  1.0436354e-10 2.8492637e-14 9.5979192e-13 3.9646077e-12 2.4118939e-13]
 [1.7448327e-06 9.9999821e-01 2.8195422e-28 1.9839173e-20 8.7925689e-29
  1.1132878e-29 4.1739516e-27 9.7473487e-29 6.2093257e-29 1.7096656e-32]
 [9.9999988e-01 4.2996430e-08 1.4718765e-10 7.4796340e-08 1.6486961e-11
  5.9200661e-10 1.6162152e-15 9.5268645e-12 1.3649127e-11 5.2558950e-13]]


In [11]:
predict = np.argmax(model.predict(X_test), axis=-1)
print(predict)

[1 1 1 0]


In [12]:
model.save('ScreamDemo1.h5') 
print('Model exported and finished')

Model exported and finished
